In [9]:
from google.colab import files
#upload JSON
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 76 bytes


# You'll need a kaggle account and the JSON.


In [12]:
!pip install kaggle
!kaggle datasets download -d andrewmvd/malignant-lymphoma-classification

 99% 1.33G/1.34G [00:08<00:00, 230MB/s]
100% 1.34G/1.34G [00:08<00:00, 178MB/s]


In [13]:
!unzip /content/malignant-lymphoma-classification.zip

Archive:  /content/malignant-lymphoma-classification.zip
replace CLL/sj-03-2810_001.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: All
  inflating: CLL/sj-03-2810_001.tif  
  inflating: CLL/sj-03-2810_002.tif  
  inflating: CLL/sj-03-2810_003.tif  
  inflating: CLL/sj-03-2810_004.tif  
  inflating: CLL/sj-03-2810_005.tif  
  inflating: CLL/sj-03-2810_006.tif  
  inflating: CLL/sj-03-2810_007.tif  
  inflating: CLL/sj-03-2810_008.tif  
  inflating: CLL/sj-03-2810_009.tif  
  inflating: CLL/sj-03-2810_010.tif  
  inflating: CLL/sj-03-2810_011.tif  
  inflating: CLL/sj-03-476_001.tif   
  inflating: CLL/sj-03-476_002.tif   
  inflating: CLL/sj-03-476_003.tif   
  inflating: CLL/sj-03-476_004.tif   
  inflating: CLL/sj-03-476_005.tif   
  inflating: CLL/sj-03-476_006.tif   
  inflating: CLL/sj-03-476_007.tif   
  inflating: CLL/sj-03-476_008.tif   
  inflating: CLL/sj-03-476_009.tif   
  inflating: CLL/sj-03-476_010.tif   
  inflating: CLL/sj-03-4957_001.tif  
  inflating: CLL/sj-03-4957_002.t

# Images are preprocessed into the following format (see MUTAG example in repo if unclear):

n = total number of nodes (in entire dataset, should be number of cells per image * number of images)

m = total number of edges (number of cell-cell connections, all images)

N = number of graphs (total number of images in the dataset)


(1) 	DS_A.txt (m lines) 
	sparse (block diagonal) adjacency matrix for all graphs,
	each line corresponds to (row, col) resp. (node_id, node_id)
  - These values on these need to be exactly match with the respective global node_id.
  eg:
  1,2 \n
  2,3 \n
  1,2 \n
  3,1 \n
  2,3 \n
  ...

(2) 	DS_graph_indicator.txt (n lines)
	column vector of graph identifiers for all nodes of all graphs,
	the value in the i-th line is the graph_id of the node with node_id i
  - e.g., we have 9 nodes across 3 graphs, each with 3 nodes:
      1
      1
      1
      2
      2
      2
      3
      3
      3

(3) 	DS_graph_labels.txt (N lines) 
	class labels for all graphs in the dataset,
	the value in the i-th line is the class label of the graph with graph_id i.
  - Should be 0,1, or 2 depending on trinary label (CLL vs FL vs MCL).
  -
  0
  0
  0
  ...
  1
  1
  1
  ...
  2
  2
  2
  ...

(4) 	DS_node_labels.txt (n lines)
	column vector of node labels,
	the value in the i-th line corresponds to the node with node_id i
  - i.e., cell type, or something similar. preferably, a tuple of R,G,B values. If not possible, default to "x" or some arbitary placeholder character.

There are OPTIONAL files if the respective information is available:

(5) 	DS_edge_labels.txt (m lines; same size as DS_A_sparse.txt)
	labels for the edges in DD_A_sparse.txt 
  - we only have one edge type, so can just put "e" or something for all.


(6) 	DS_edge_attributes.txt (m lines; same size as DS_A.txt)
	attributes for the edges in DS_A.txt 

  - Should be pairwise distances between cells, for this project, as a float. If all images not the same dimensions or scale, then STANDARDIZE this single feature for each image independently (via sci-kit.preprocessing or similar) learn such that each value is on a (0,1) scale for its own image.

(7) 	DS_node_attributes.txt (n lines) 
	matrix of node attributes,
	the comma seperated values in the i-th line is the attribute vector of the node with node_id i
  - put R,G,B pixel intensities normalized to (0,255) if not already.
  - Future work: can also threshold edge distances and then use node-degree as another node attribute.



Now we can load these into pytorch-geometric...